In [1]:
import sys
sys.path.append('../material/')
from TicTacToe import Board
import numpy as np

In [2]:
# tree node class definition
class TreeNode():
    def __init__(self, board, parent):
        # init associated board state
        self.board = board
        
        # init is node terminal flag
        if self.board.is_win() or self.board.is_draw():
            # we have a terminal node
            self.is_terminal = True
        
        # otherwise
        else:
            # we have a non-terminal node
            self.is_terminal = False
        self.is_fully_expanded = self.is_terminal
        self.parent = parent
        self.visits = 0
        self.score = 0
        self.children = {}

In [12]:
# MCTS class definition
class MCTS_POMDP():
    # search for the best move in the current position
    def search(self, initial_state, iteration=3000):
        # create root node
        self.root = TreeNode(initial_state, None)

        # walk through 1000 iterations
        for iteration in range(iteration):
            # select a node (selection phase)
            node = self.select(self.root)
            
            # scrore current node (simulation phase)
            score = self.simulation(node.board)
            
            # backpropagate results
            self.backpropagate(node, score)
        
        # pick up the best move in the current position
        try:
            return self.get_best_move(self.root, 0)
        
        except:
            pass
    
    # select most promising node
    def select(self, node):
        # make sure that we're dealing with non-terminal nodes
        while not node.is_terminal:
            # case where the node is fully expanded
            if node.is_fully_expanded:
                node = self.get_best_move(node, 2)
            
            # case where the node is not fully expanded 
            else:
                # otherwise expand the node
                return self.expand(node)
       # return node
        return node

    # expand node
    def expand(self, node):
        # generate legal states (moves) for the given node
        states = node.board.generate_states()
        
        # loop over generated states (moves)
        for state in states:
            # make sure that current state (move) is not present in child nodes
            if str(state.position) not in node.children:
                # create a new node
                new_node = TreeNode(state, node)
                
                # add child node to parent's node children list (dict)
                node.children[str(state.position)] = new_node
                
                # case when node is fully expanded
                if len(states) == len(node.children):
                    node.is_fully_expanded = True
                
                # return newly created node
                return new_node
        
        # debugging
        print('접근 안되는 구간입니다!!!') # 디버그 용도

    # simulate the game via making random moves until reach end of the game
    def simulation(self, board):
        # make random moves for both sides until terminal state of the game is reached
        while not board.is_win():
            # try to make a move
            try:
                # make the on board
                board = np.random.choice(board.generate_states())
                
            # no moves available
            except:
                # return a draw score
                return 0
        
        # return score from the player "x" perspective
        if board.player_2 == 'x': return 1
        elif board.player_2 == 'o': return -1

    # backpropagate the number of visits and score up to the root node
    def backpropagate(self, node, score, gamma=0.99):
        # update nodes's up to root node
        while node is not None:
            # update node's score
            node.score = gamma*node.score*node.visits + score
            node.score /= (node.visits+1)
            
            # update node's visits
            node.visits += 1
            
            # update node's score
            #node.score += score
            
            # set node to parent
            node = node.parent
            
    # select the best node basing on UCB1 formula
    def get_best_move(self, node, C_const=2):
        best_score = -np.inf
        best_moves = []

        for child_node in node.children.values():
            # define current_player
            if child_node.board.player_2 == 'x': current_player = 1
            elif child_node.board.player_2 == 'o': current_player = -1
            
            # get move score using UCB formula
            #move_score = current_player*child_node.score/child_node.visits+\
            move_score = current_player*child_node.score+\
                C_const*np.sqrt(np.log(node.visits/child_node.visits))

            # better move has been found
            if move_score > best_score:
                best_score = move_score
                best_moves = [child_node]

            # found as good move as alread avilable
            elif move_score == best_score:
                best_moves.append(child_node)

        # return one of the best moves randomly
        return np.random.choice(best_moves)

In [13]:
board = Board()
root = TreeNode(board,None)
#root = TreeNode(None)
root.children['child_1']=TreeNode(board.make_move(1,1),root)
print(root.__dict__)
print(root.children['child_1'].__dict__)

{'board': <TicTacToe.Board object at 0x7ff35a018040>, 'is_terminal': False, 'is_fully_expanded': False, 'parent': None, 'visits': 0, 'score': 0, 'children': {'child_1': <__main__.TreeNode object at 0x7ff35a433dc0>}}
{'board': <TicTacToe.Board object at 0x7ff35a433700>, 'is_terminal': False, 'is_fully_expanded': False, 'parent': <__main__.TreeNode object at 0x7ff35a07fe20>, 'visits': 0, 'score': 0, 'children': {}}


In [15]:
board = Board()
mcts = MCTS_POMDP()
board.game_loop(mcts)

  
Tic Tac Toe 게임을 시작합니다.
   순서는 1,2처럼 타이핑하세요. 1은 "행", 2는 "열"을 의미합니다.
   "exit"은 게임을 종료합니다.

---------------
 "x" 의 차례: 
---------------

 .  .  . 
 .  .  . 
 .  .  . 

>> 1,1

---------------
 "x" 의 차례: 
---------------

 x  .  . 
 .  o  . 
 .  .  . 

>> 3,1

---------------
 "x" 의 차례: 
---------------

 x  .  . 
 o  o  . 
 x  .  . 

>> 2,3

---------------
 "x" 의 차례: 
---------------

 x  o  . 
 o  o  x 
 x  .  . 

>> 1,3

---------------
 "x" 의 차례: 
---------------

 x  o  x 
 o  o  x 
 x  o  . 

 게임의 승자: "o"!
